This notebook is part of the $\omega radlib$ documentation: https://docs.wrl.vis.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xarray GAMIC backend

In this example, we read GAMIC (HDF5) data files using the xradar `gamic` backend.

In [ ]:
import glob
import wradlib as wrl
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import xradar as xd
import datatree as xt
import xarray as xr

try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    plt.ion()

## Load GAMIC HDF5 Volume Data

In [ ]:
fpath = "hdf5/DWD-Vol-2_99999_20180601054047_00.h5"
f = wrl.util.get_wradlib_data_file(fpath)
vol = xd.io.open_gamic_datatree(f)

## Inspect RadarVolume

In [ ]:
display(vol)

## Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

## Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol["sweep_0"])

## Georeferencing

In [ ]:
swp = vol["sweep_0"].ds.copy()
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)

In [ ]:
swp = swp.wrl.georef.georeference()
display(swp)

## Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by ODIM_H5 standard.

In [ ]:
display(swp.DBZH)

## Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

For more details on plotting radar data see under [Visualization](../../plotting.ipynb).

In [ ]:
swp.DBZH.sortby("time").plot(x="range", y="time", add_labels=False)

In [ ]:
fig = plt.figure(figsize=(10, 10))
pm = swp.DBZH.wrl.vis.plot(crs={"latmin": 3e3}, fig=fig)

## Retrieve explicit group


In [ ]:
swp_b = xr.open_dataset(
    f, engine="gamic", group="sweep_9", backend_kwargs=dict(reindex_angle=False)
)
display(swp_b)